In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from datasets import load_dataset
import re
import os

In [9]:
dataset = load_dataset("squad_v2")

Found cached dataset squad_v2 (/home/hp/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [59]:
dataset_train = dataset['train']
dataset_val = dataset['validation']


In [27]:
# Create a custom sample in the squad format
def add_squad_sample(samples, title, context, question, answer):
    if answer not in context:
        print(f'Could not find answer({answer}) in context')
        return samples
    answer_start = context.index(answer)
    id = "abcdefg"  # IDK what to do with the id here
    d =  {
        'id': id,
        'title': title,
        'context': context,
        'question': question,
        'answers': {
            'text': [answer],
            'answer_start': [answer_start]
        }
    }
    samples.append(d)
    return samples

add_squad_sample([],"Programming", "Python programming is fun.", "What is Python?", "fun.")

[{'id': 'abcdefg',
  'title': 'Programming',
  'context': 'Python programming is fun.',
  'question': 'What is Python?',
  'answers': {'text': ['fun.'], 'answer_start': [22]}}]

In [39]:
# Read and clean the context file
def clean_context(filename):
    with open(filename, 'r', encoding="utf8") as f:
        text = f.read()
    text = re.sub("\\n", ' ',text, re.ASCII)
    text = re.sub("\s{2,}", ' ',text, re.ASCII)
    text = re.sub("“|”", '"',text, re.ASCII)
    text = re.sub("‘|’", "'",text, re.ASCII)
    text = re.sub("\"", '\\"',text, re.ASCII)
    text = re.sub("_", '',text, re.ASCII)
    text = re.sub("\s{2,}", ' ',text, re.ASCII)
    text = text.strip()
    return text


---
# Create the custom dataset

In [55]:
samples = []

########################################################################
# Protagonist Questions
context = clean_context(os.path.join('text', 'protagonist-first-paragraph.txt'))
samples = add_squad_sample(samples,
    title = "The Hound of the Baskervilles",
    context = context,
    question = 'What is a characteristic of Sherlock Holmes?',
    answer = 'usually very late in the mornings'
)

########################################################################
# Crime Scene Questions
context = clean_context(os.path.join('text', 'crimescene.txt'))
samples = add_squad_sample(samples,
    title = "The Hound of the Baskervilles",
    context = context,
    question = 'What habit does Sir Charles of Baskerville have?',
    answer = 'in the habit every night before going to bed of walking down the famous yew alley of Baskerville Hall'
)
samples = add_squad_sample(samples,
    title = "The Hound of the Baskervilles",
    context = context,
    question = 'What was Sir Charles doing the night of his murder?',
    answer = 'That night he went out as usual for his nocturnal walk, in the course of which he was in the habit of smoking a cigar'
)
samples = add_squad_sample(samples,
    title = "The Hound of the Baskervilles",
    context = context,
    question = "How was Sir Charles's body found?",
    answer = "Charles's footmarks were easily traced down the alley."
)
samples = add_squad_sample(samples,
    title = "The Hound of the Baskervilles",
    context = context,
    question = "Did anyone hear what happened to Sir Charles?",
    answer = "heard cries but is unable to state from what direction they came"
)
samples = add_squad_sample(samples,
    title = "The Hound of the Baskervilles",
    context = context,
    question = "Was there anything interesting about the victims body?",
    answer = "No signs of violence were to be discovered upon Sir Charles's person, and though the doctor's evidence pointed to an almost incredible facial distortion"
)
context = clean_context(os.path.join('text', 'crimescene2.txt'))
samples = add_squad_sample(samples,
    title = "The Hound of the Baskervilles",
    context = context,
    question = 'How did Sir Charles die?',
    answer = 'affection of the heart'
)
########################################################################
# Antagonist Questions
context = clean_context(os.path.join('text', 'stapleton.txt'))
samples = add_squad_sample(samples,
   title = "The Hound of the Baskervilles",
   context = context,
   question = 'What ideology doe Stapleton follow?',
   answer = 'naturalist.'
)

########################################################################
# Resolution Questions
context = clean_context(os.path.join('text', 'stapleton-murder.txt'))
samples = add_squad_sample(samples,
   title = "The Hound of the Baskervilles",
   context = context,
   question = 'How did Stapleton kill Sir Charles?',
   answer = 'get his hound, to treat it with his infernal paint, and to bring the beast round to the gate at which he had reason to expect that he would find the old gentleman waiting. The dog, incited by its master, sprang over the wicket-gate and pursued the unfortunate baronet, who fled screaming down the yew alley. In that gloomy tunnel it must indeed have been a dreadful sight to see that huge black creature, with its flaming jaws and blazing eyes, bounding after its victim.'
)


In [54]:
print(samples)

[{'id': 'abcdefg', 'title': 'The Hound of the Baskervilles', 'context': 'Mr. Sherlock Holmes, who was usually very late in the mornings, save upon those not infrequent occasions when he was up all night, was seated at the breakfast table. I stood upon the hearth-rug and picked up the stick which our visitor had left behind him the night before. It was a fine, thick piece of wood, bulbous-headed, of the sort which is known as a \\"Penang lawyer.\\" Just under the head was a broad silver band nearly an inch across. \\"To James Mortimer, M.R.C.S., from his friends of the C.C.H.,\\" was engraved upon it, with the date \\"1884.\\" It was just such a stick as the old-fashioned family practitioner used to carry—dignified, solid, and reassuring. \\"Well, Watson, what do you make of it?\\" Holmes was sitting with his back to me, and I had given him no sign of my occupation. \\"How did you know what I was doing? I believe you have eyes in the back of your head.\\" \\"I have, at least, a well-pol

In [70]:
n_rows = dataset_train.num_rows
n_samples = len(samples)
dataset_train_custom = dataset_train

130319


In [71]:
dir(dataset_train)

for sample in samples:
    dataset_train_custom = dataset_train_custom.add_item(sample)


In [67]:
dataset_train

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [ ]:
dataset_train_custom

In [74]:
dataset_train_custom[130327]

{'id': 'abcdefg',
 'title': 'The Hound of the Baskervilles',
 'context': '\\"But it was some time coming. The old gentleman could not be decoyed outside of his grounds at night. Several times Stapleton lurked about with his hound, but without avail. It was during these fruitless quests that he, or rather his ally, was seen by peasants, and that the legend of the demon dog received a new confirmation. He had hoped that his wife might lure Sir Charles to his ruin, but here she proved unexpectedly independent. She would not endeavour to entangle the old gentleman in a sentimental attachment which might deliver him over to his enemy. Threats and even, I am sorry to say, blows refused to move her. She would have nothing to do with it, and for a time Stapleton was at a deadlock. \\"He found a way out of his difficulties through the chance that Sir Charles, who had conceived a friendship for him, made him the minister of his charity in the case of this unfortunate woman, Mrs. Laura Lyons. By 

In [75]:
dataset_train_custom[0]


{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}